In [13]:
# Library Import

import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

In [14]:
# Dense Layer

class Layer_Dense:
    # Layer initialization
    # Initialize weights (small random) and biases (zero)
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [15]:
# Activation Functions

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [16]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [17]:
# Start of Optimizers

class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0., momentum=0., adaptive=False):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.adaptive = adaptive


    def pre_update_params(self):
        # Adjust learning rate if decay is used
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        # Initialize momentum or adaptive caches if not already present
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
        if self.adaptive:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)

        # Momentum update
        if self.momentum:
            weight_updates = (self.momentum * layer.weight_momentums) - \
                             (self.current_learning_rate * layer.dweights)
            bias_updates = (self.momentum * layer.bias_momentums) - \
                           (self.current_learning_rate * layer.dbiases)
            layer.weight_momentums = weight_updates
            layer.bias_momentums = bias_updates

        # Adagrad update
        elif self.adaptive:
            # Accumulate squared gradients
            layer.weight_cache += layer.dweights ** 2       
            layer.bias_cache += layer.dbiases ** 2
            # Avoid division by zero
            weight_updates = -self.current_learning_rate * layer.dweights / \
                             (np.sqrt(layer.weight_cache) + 1e-7)  
            bias_updates = -self.current_learning_rate * layer.dbiases / \
                           (np.sqrt(layer.bias_cache) + 1e-7)

        # Vanilla SGD update
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        # Apply updates to the layer
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        # Increment iteration count after all layers are updated
        self.iterations += 1

In [18]:
# Create the dataset

X, y = spiral_data(samples = 100, classes = 3)

print(X[:5])
print(X.shape)
print(y[:5])
print(y.shape)

[[0.         0.        ]
 [0.00299556 0.00964661]
 [0.01288097 0.01556285]
 [0.02997479 0.0044481 ]
 [0.03931246 0.00932828]]
(300, 2)
[0 0 0 0 0]
(300,)


In [19]:
# Neural Network initialization

# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)
# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()
# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)
# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()
# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

In [20]:
# Training function

def train_network(optimizer, epochs=1000, title="Optimizer"):
    print(f"\n{'='*60}\n{title}\n{'='*60}")

    # Store loss and accuracy history
    losses = []
    accuracies = []

    for epoch in range(epochs):
        # Forward pass
        # Input to first dense layer
        dense1.forward(X)                 
        # ReLU activation
        activation1.forward(dense1.output) 
        # Second dense layer  
        dense2.forward(activation1.output) 
        # Softmax activation
        activation2.forward(dense2.output) 


        # Compute loss
        loss = np.mean(loss_function.forward(activation2.output, y))
        # Save loss history
        losses.append(loss)               

        # Predictions & accuracy
        predictions = np.argmax(activation2.output, axis=1)
        # Predicted classes 
        if len(y.shape) == 2:
            # Convert one-hot
            y_eval = np.argmax(y, axis=1)                   
        else:
            y_eval = y
        # Accuracy
        accuracy = np.mean(predictions == y_eval) 
        # Save accuracy history          
        accuracies.append(accuracy)                        

        # Print every 100 epochs
        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch:4d}  -  Learning Rate: {optimizer.current_learning_rate:.4f} "
                  f" -  Loss: {loss:.4f}  -  Accuracy: {accuracy:.4f}")

        # Backward pass
        # Gradient from loss
        loss_function.backward(activation2.output, y)       
        dvalues = loss_function.dinputs

        if epoch == 0:
            # Inspect gradient shape
            print(f"\n[dvalues shape: {dvalues.shape}]")   

        # Softmax backward
        activation2.backward(dvalues)   
        # Dense layer 2 backward                   
        dense2.backward(activation2.dinputs)    
        # ReLU backward          
        activation1.backward(dense2.dinputs)     
        # Dense layer 1 backward         
        dense1.backward(activation1.dinputs)       

        # Update weights
        # Apply learning rate decay
        optimizer.pre_update_params()   
        # Update layer 1                   
        optimizer.update_params(dense1)   
        # Update layer 2                
        optimizer.update_params(dense2)      
        # Increment optimizer iteration             
        optimizer.post_update_params()                     

    print(f"\n Training completed for {title}\n")

    return losses, accuracies



In [21]:
# Learning Rate Decay Optimizer

opt_sgd = Optimizer_SGD(learning_rate=1.0, decay=1e-3)
losses_sgd, accuracies_sgd = train_network(opt_sgd, title="Learning Rate Decay")



Learning Rate Decay
Epoch    0  -  Learning Rate: 1.0000  -  Loss: 1.0986  -  Accuracy: 0.3400

[dvalues shape: (300, 3)]
Epoch  100  -  Learning Rate: 0.9099  -  Loss: 1.0983  -  Accuracy: 0.3767
Epoch  200  -  Learning Rate: 0.8340  -  Loss: 1.0877  -  Accuracy: 0.3767
Epoch  300  -  Learning Rate: 0.7698  -  Loss: 1.0836  -  Accuracy: 0.4100
Epoch  400  -  Learning Rate: 0.7148  -  Loss: 1.0836  -  Accuracy: 0.4100
Epoch  500  -  Learning Rate: 0.6671  -  Loss: 1.0835  -  Accuracy: 0.4033
Epoch  600  -  Learning Rate: 0.6254  -  Loss: 1.0832  -  Accuracy: 0.4167
Epoch  700  -  Learning Rate: 0.5886  -  Loss: 1.0825  -  Accuracy: 0.4333
Epoch  800  -  Learning Rate: 0.5559  -  Loss: 1.0815  -  Accuracy: 0.4033
Epoch  900  -  Learning Rate: 0.5266  -  Loss: 1.0809  -  Accuracy: 0.3967
Epoch  999  -  Learning Rate: 0.5005  -  Loss: 1.0804  -  Accuracy: 0.3933

 Training completed for Learning Rate Decay



In [22]:
# Momentum Optimizer

opt_momentum = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.9)
losses_mom, accuracies_mom = train_network(opt_momentum, title="Momentum")



Momentum
Epoch    0  -  Learning Rate: 1.0000  -  Loss: 1.0804  -  Accuracy: 0.3933

[dvalues shape: (300, 3)]
Epoch  100  -  Learning Rate: 0.9099  -  Loss: 1.0796  -  Accuracy: 0.3767
Epoch  200  -  Learning Rate: 0.8340  -  Loss: 1.0721  -  Accuracy: 0.3800
Epoch  300  -  Learning Rate: 0.7698  -  Loss: 1.0594  -  Accuracy: 0.4367
Epoch  400  -  Learning Rate: 0.7148  -  Loss: 1.0567  -  Accuracy: 0.4400
Epoch  500  -  Learning Rate: 0.6671  -  Loss: 1.0516  -  Accuracy: 0.4367
Epoch  600  -  Learning Rate: 0.6254  -  Loss: 1.0292  -  Accuracy: 0.4433
Epoch  700  -  Learning Rate: 0.5886  -  Loss: 1.0259  -  Accuracy: 0.4533
Epoch  800  -  Learning Rate: 0.5559  -  Loss: 1.0249  -  Accuracy: 0.4467
Epoch  900  -  Learning Rate: 0.5266  -  Loss: 1.0245  -  Accuracy: 0.4400
Epoch  999  -  Learning Rate: 0.5005  -  Loss: 1.0243  -  Accuracy: 0.4467

 Training completed for Momentum



In [ ]:
# Adaptive Gradient Optimizer

opt_adagrad = Optimizer_SGD(learning_rate=1.0, decay=1e-3, adaptive=True)
losses_adagrad, accuracies_adagrad = train_network(opt_adagrad, title="Adaptive Gradient")


Adaptive Gradient
Epoch    0  -  Learning Rate: 1.0000  -  Loss: 1.0243  -  Accuracy: 0.4467

[dvalues shape: (300, 3)]
Epoch  100  -  Learning Rate: 0.9099  -  Loss: 1.0244  -  Accuracy: 0.4200
Epoch  200  -  Learning Rate: 0.8340  -  Loss: 1.0235  -  Accuracy: 0.4367
Epoch  300  -  Learning Rate: 0.7698  -  Loss: 1.0234  -  Accuracy: 0.4400
Epoch  400  -  Learning Rate: 0.7148  -  Loss: 1.0232  -  Accuracy: 0.4467
Epoch  500  -  Learning Rate: 0.6671  -  Loss: 1.0232  -  Accuracy: 0.4500
Epoch  600  -  Learning Rate: 0.6254  -  Loss: 1.0231  -  Accuracy: 0.4467
Epoch  700  -  Learning Rate: 0.5886  -  Loss: 1.0231  -  Accuracy: 0.4500
Epoch  800  -  Learning Rate: 0.5559  -  Loss: 1.0230  -  Accuracy: 0.4500
Epoch  900  -  Learning Rate: 0.5266  -  Loss: 1.0230  -  Accuracy: 0.4500
Epoch  999  -  Learning Rate: 0.5005  -  Loss: 1.0230  -  Accuracy: 0.4500

 Training completed for Adaptive Gradient



In [26]:
# Store results for each optimizer: (name, loss history, accuracy history)

results = [
    ("Learning Rate Decay", losses_sgd, accuracies_sgd),
    ("Momentum", losses_mom, accuracies_mom),
    ("Adaptive Gradient", losses_adagrad, accuracies_adagrad)
]

print("\n Optimizer Comparison Summary with Stabilizer set at 1e-6")

# Loop through each optimizer's results
for name, losses, accuracies in results:
    # Determine the epoch where loss change between consecutive epochs is very small 
    # This is considered the "stabilized" epoch
    stabilized_epoch = next(
        # Set stabilization to 1e-6
        (i for i in range(1, len(losses)) if abs(losses[i] - losses[i - 1]) < 1e-6),
        len(losses) 
    )
    
    # Take the final accuracy after training
    final_acc = accuracies[-1] if len(accuracies) > 0 else 0
    
    # Print optimizer name, stabilized epoch, and final accuracy
    print(f"{name:20s}  -  Stabilized at epoch: {stabilized_epoch:4d}  -  Final Accuracy: {final_acc:.4f}")


 Optimizer Comparison Summary with Stabilizer set at 1e-6
Learning Rate Decay   -  Stabilized at epoch:    1  -  Final Accuracy: 0.3933
Momentum              -  Stabilized at epoch:   54  -  Final Accuracy: 0.4467
Adaptive Gradient     -  Stabilized at epoch:  122  -  Final Accuracy: 0.4500
